<a href="https://colab.research.google.com/github/Ismail-Armutcu/Algorithms-for-Interactive-Sytems/blob/main/MMI513_PA4_Tournaments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MMI513 Programming Assignment #4**
## **Tournaments - King of the Hill**
- İsmail Hakkı Armutcu
- 2374395



In [232]:
import numpy as np
import math

In [233]:
class Player: #Player  class to implement tournaments such as pyramid match and king of the hill

  def __init__(self, name, rank = 0,peerStatus = True): #Constructor
    assert(type(peerStatus) == bool)
    assert(rank == None or rank >=0)
    self.name = name
    self.rank = rank
    self.peerWinner = peerStatus

  def __str__(self):  #To print the class
    return f"Name:{self.name} Rank:{self.rank} PeerWinner:{self.peerWinner}"

  def __repr__(self): #The string representation of class
    return str(self)

  def __eq__(self, object2): #Equality test for the class
    if(type(object2) == Player):
      return self.name == object2.name
    elif(type(object2) == str):
      return self.name == object2
    else:
      return False

  def __ne__(self, object2): # Not equal test for the class
    return self.name != object2.name

  def setRank(self,rank):  # Sets the rank of the object
    assert( rank >=0 or rank == None)
    self.rank = rank

  def getRank(self):    # Returns the rank of the object
    return self.rank

  def setPeerWinner(self, status): # Sets the peerWinner status of the object
    assert(type(status) == bool)
    self.peerWinner = status

  def getPeerWinner(self): # Returns the peerWinner status of the object
    return self.peerWinner



def printPlayers(P): ## Prints the list of players in an organized manner
  names = []
  ranks = []
  peerWinner = []
  for p in P:
    names.append(p.name)
    ranks.append(p.getRank())
    peerWinner.append(p.getPeerWinner())
  print("Names:     ",names)
  print("Ranks:     ",ranks)
  print("PeerWinner:",peerWinner)


In [234]:
def unrankedplayers(n): #Create a list of n players with ranks none
    P = []
    for ind in range(n):
        p = Player(name = 'P' + str(ind), rank = None, peerStatus = True)
        P.append(p)

    return P

def enumeration(P):   # Enumerate (shuffle) the list of players
    assert(type(P[0]) == Player)
    lst = np.arange(len(P))
    order = np.random.permutation(lst)
    M = []
    for ind in order:
      M.append(P[ind])
    return M

def rankeds(P, r):  # From the list of players return those with ranks n
    assert(type(P[0]) == Player)
    assert(r >= 0)
    W = []
    for p in P:
      if p.getRank() == r:
        W.append(p)
    return W

def match(p, q, thr, tieflag=False): ## Determine the winner of two players
    toss = np.random.rand(1)
    upC = 1
    downC = 1
    if tieflag: # If Tie is allowed
      upC = 1.1
      downC = 0.9
    if thr * downC < toss < thr * upC: # If the random number is around thr, result is a tie
        result = Player("None",None,False) # Tie
    elif toss > thr * upC:
        result = p
    else:
        result = q
    return result

def ladder_match(P, p, q):
    m = match(p, q, 0.5)
    if m == "None" or m == p:
        return P
    else:
        indp = P.index(p)
        indq = P.index(q)
        rankp = P[indp].getRank()
        rankq = P[indq].getRank()
        if rankq == None:
            rankq = rankp
        P[indp].setRank(rankq)
        P[indq].setRank(rankp)
        return P

def initial_rank_adjustment(P, S): ## Se the rank of players in the list according to input S
    assert len(P) == sum(S)
    m = len(S)
    R = P.copy()
    R = enumeration(R)
    for ind in range(S[0]):
      R[ind].setRank(0) # The initial champion(s)
    c =  S[0]
    for rank in range(1, m):
        W = rankeds(R, rank - 1) # The runners-up
        Mp = enumeration(W)
        for ind in range(S[rank]):
            R[c+ind].setRank(rank)
            jnd = ind % len(Mp)

            if Mp[jnd].getRank() != rank:
              R = ladder_match(R, Mp[jnd], R[c + ind])
        c += S[rank]
    return R

## Test Player Class implementations of
- unrankedplayers()
- enumeration()
- initial_rank_adjustment()

In [235]:
P = unrankedplayers(11)
print("unRankedPlayers")
printPlayers(P)



S = [1, 2, 3, 5] # Ranking structure: One champion, 2 second places, etc.
M = enumeration(P)
print("Enumeration")
printPlayers(M)
R = initial_rank_adjustment(P, S)
print("Initial Rank Adjustment")
printPlayers(R)



unRankedPlayers
Names:      ['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10']
Ranks:      [None, None, None, None, None, None, None, None, None, None, None]
PeerWinner: [True, True, True, True, True, True, True, True, True, True, True]
Enumeration
Names:      ['P0', 'P3', 'P10', 'P6', 'P7', 'P9', 'P2', 'P4', 'P5', 'P8', 'P1']
Ranks:      [None, None, None, None, None, None, None, None, None, None, None]
PeerWinner: [True, True, True, True, True, True, True, True, True, True, True]
Initial Rank Adjustment
Names:      ['P2', 'P1', 'P10', 'P8', 'P9', 'P5', 'P4', 'P0', 'P7', 'P6', 'P3']
Ranks:      [3, 0, 2, 3, 1, 1, 3, 2, 3, 2, 3]
PeerWinner: [True, True, True, True, True, True, True, True, True, True, True]


In [236]:
def pyramid_match(P, p, q):

  pi = P.index(p)   ## Find the players p and q in list P
  qi = P.index(q)
  playerP = P[pi]
  playerQ = P[qi]

  #print((playerP.getRank() == playerQ.getRank()) and (not playerQ.getPeerWinner()))
  #print((playerP.getRank() == playerQ.getRank()-1) and playerQ.getPeerWinner())

  if not ((playerP.getRank() == playerQ.getRank()) and (not playerQ.getPeerWinner())) | ((playerP.getRank() == playerQ.getRank()-1) and playerQ.getPeerWinner()):
    print('No match was played! Ranks do not match the conditions for pyramid match!')
    return P

  R = P.copy() # We copy the player dictionary first
  R.pop(R.index(playerP)) #Find player p in list and pop it
  R.pop(R.index(playerQ)) #Find player q in list and pop it


  # We do not have the two players in our set of players now.
  m = match(playerP, playerQ, 0.5) # Let them compete

  if (playerP.getRank() == playerQ.getRank()): # Peer match
    if (((m == p) and (not playerP.getPeerWinner())) or (((m == q) or m == "None") and playerP.getPeerWinner())):
      playerPP = Player(playerP.name,playerP.getRank(),playerP == m)
    else:
      playerPP = playerP
    if (m == q):
      playerQP = Player(playerQ.name, playerQ.getRank(), True)
    else:
       playerQP = playerQ

    R.append(playerPP) ## add the updated players in the list
    R.append(playerQP)
    return R
  else: # Rank challenge match
    playerQP = Player(playerQ.name, playerQ.getRank(), False)
    if (m == p) or m == "None":
      R.append(playerP)
      R.append(playerQP)  # add the updated players in the list
      return R
    else:
      playerPP = Player(playerP.name, playerQ.getRank(), False)
      playerQP.setRank(playerP.getRank())
      R.append(playerPP)
      R.append(playerQP)  # add the updated players in the list
      return R

## Arrange the list of players
- Create list of n players.
- Set the ranks
- Set the peerWinner status as true

In [237]:
n = 11
P = unrankedplayers(n)
S = [1, 2, 3, 5]
R = initial_rank_adjustment(P, S)
# Appending the peerWinner field to R makes it into pyramid structure
for r in R:
  r.setPeerWinner(True)
printPlayers(R)

Names:      ['P0', 'P5', 'P2', 'P1', 'P8', 'P10', 'P9', 'P7', 'P6', 'P3', 'P4']
Ranks:      [3, 0, 2, 3, 1, 1, 3, 3, 3, 2, 2]
PeerWinner: [True, True, True, True, True, True, True, True, True, True, True]


## Test the player class implementation of pyramid_match

In [238]:
R = pyramid_match(R, "P6", "P9")
printPlayers(R)

No match was played! Ranks do not match the conditions for pyramid match!
Names:      ['P0', 'P5', 'P2', 'P1', 'P8', 'P10', 'P9', 'P7', 'P6', 'P3', 'P4']
Ranks:      [3, 0, 2, 3, 1, 1, 3, 3, 3, 2, 2]
PeerWinner: [True, True, True, True, True, True, True, True, True, True, True]


## King of the Hill Tournament implementation with Player Class

In [239]:
def kingofthehill(P = unrankedplayers(7)): ## The input is list of players
  assert(len(P) >= 1)                       #Check if the list of players is non empty
  assert(math.log2(len(P)+1).is_integer())  # Check if n+1 is a power of two
  assert(type(P[0]) == Player)              # Check if the list consists of Player objects

  m = int(math.log2(len(P)+1))            # m = log(n+1)
  S = []
  for i in range(m):
    S.append(2**i)    ## form the ranks

  R = initial_rank_adjustment(P,S) ## Initialize the ranks
  print("Players Initialized")
  printPlayers(R)
  for rp in range(1,m):
    r = (m-1)-(rp-1)
    M = enumeration(rankeds(R,r))
    l = len(M)

    for i in range(l//2):
      M[2*i].setPeerWinner(False)
      M[2*i+1].setPeerWinner(False)
      print("Players face off before")
      printPlayers([M[2*i],  M[2*i+1]])
      R = pyramid_match(R,M[2*i].name,M[2*i+1].name)
      print("Players face off after")
      ind1 = R.index(M[2*i])
      ind2 = R.index(M[2*i+1])
      printPlayers([R[ind1],  R[ind2]])

    M = []
    for p in rankeds(R,r):
      if(p.getPeerWinner() == True):
        M.append(p)
    Mp = enumeration(rankeds(R,r-1))
    for i in range(l//2):
      print("Players face off before")
      printPlayers([Mp[i],M[i]])
      R = pyramid_match(R,Mp[i].name,M[i].name)
      print("Players face off after")
      ind1 = R.index(Mp[i])
      ind2 = R.index(M[i])
      printPlayers([R[ind1],  R[ind2]])

  return R


## Test the King of the Hill Tournament implementation with Player Class

In [240]:

R = kingofthehill()
print("Tournament is complete")
printPlayers(R)

Players Initialized
Names:      ['P1', 'P2', 'P4', 'P3', 'P0', 'P6', 'P5']
Ranks:      [2, 0, 2, 2, 2, 1, 1]
PeerWinner: [True, True, True, True, True, True, True]
Players face off before
Names:      ['P1', 'P3']
Ranks:      [2, 2]
PeerWinner: [False, False]
Players face off after
Names:      ['P1', 'P3']
Ranks:      [2, 2]
PeerWinner: [True, False]
Players face off before
Names:      ['P4', 'P0']
Ranks:      [2, 2]
PeerWinner: [False, False]
Players face off after
Names:      ['P4', 'P0']
Ranks:      [2, 2]
PeerWinner: [True, False]
Players face off before
Names:      ['P5', 'P1']
Ranks:      [1, 2]
PeerWinner: [True, True]
Players face off after
Names:      ['P5', 'P1']
Ranks:      [1, 2]
PeerWinner: [True, False]
Players face off before
Names:      ['P6', 'P4']
Ranks:      [1, 2]
PeerWinner: [True, True]
Players face off after
Names:      ['P6', 'P4']
Ranks:      [1, 2]
PeerWinner: [True, False]
Players face off before
Names:      ['P6', 'P5']
Ranks:      [1, 1]
PeerWinner: [False, 